In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *


Using TensorFlow backend.


In [2]:
# 这是一个用来对预训练模型进行fine tune的方法，因为在项目中涉及到很多次fine-tune,为了方便，编写如下方法。
# 参数MODEL：预训练模型名称；image_size：输入数据尺寸；weights_name：fine—tune后参数保存的文件名；trainable_layers:放开训练的层数；
# lambda_func：预训练模型自带数据处理方法。
def fine_tuning(MODEL,image_size,weights_name,trainable_layers=0,lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    if trainable_layers==0:
        for layer in base_model.layers:
            layer.trainable = False
    else:
        for layer in base_model.layers[:-trainable_layers]:
            layer.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    print(x.shape)
    x = Dense(1, activation='sigmoid')(x)
    print(x.shape)
    model = Model(base_model.input, x)
    model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    gen = ImageDataGenerator() 
    train_generator = gen.flow_from_directory("train2", image_size, batch_size=100,class_mode='binary')
    validation_generator = gen.flow_from_directory("validation", image_size, batch_size=100,class_mode='binary')
    model.fit_generator(train_generator,
                    steps_per_epoch=20000/100,
                    epochs=5,
                    validation_data=validation_generator,
                    validation_steps=5000/100)
    model.save_weights(weights_name)

In [10]:
fine_tuning(ResNet50,(224,224),"w_resnet_0.h5")

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
313/313 [==============================] - 685s - loss: 0.0844 - acc: 0.9679 - val_loss: 0.7369 - val_acc: 0.9116
Epoch 2/5
313/313 [==============================] - 675s - loss: 0.0296 - acc: 0.9894 - val_loss: 0.3671 - val_acc: 0.9236
Epoch 3/5
313/313 [==============================] - 675s - loss: 0.0201 - acc: 0.9933 - val_loss: 0.2532 - val_acc: 0.9604
Epoch 4/5
313/313 [==============================] - 674s - loss: 0.0109 - acc: 0.9963 - val_loss: 0.0650 - val_acc: 0.9824
Epoch 5/5
313/313 [==============================] - 675s - loss: 0.0072 - acc: 0.9975 - val_loss: 0.1505 - val_acc: 0.9712


In [14]:
fine_tuning(ResNet50,(224,224),"w_resnet_0.h5",5)

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
313/313 [==============================] - 253s - loss: 0.1142 - acc: 0.9569 - val_loss: 0.0635 - val_acc: 0.9780
Epoch 2/5
313/313 [==============================] - 250s - loss: 0.0700 - acc: 0.9728 - val_loss: 0.0630 - val_acc: 0.9756
Epoch 3/5
313/313 [==============================] - 251s - loss: 0.0582 - acc: 0.9786 - val_loss: 0.0688 - val_acc: 0.9762
Epoch 4/5
313/313 [==============================] - 251s - loss: 0.0480 - acc: 0.9819 - val_loss: 0.0524 - val_acc: 0.9794
Epoch 5/5
313/313 [==============================] - 251s - loss: 0.0438 - acc: 0.9834 - val_loss: 0.0615 - val_acc: 0.9802


In [16]:
fine_tuning(ResNet50,(224,224),"w_resnet_0.h5",8)

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
313/313 [==============================] - 259s - loss: 0.0914 - acc: 0.9644 - val_loss: 0.0644 - val_acc: 0.9794
Epoch 2/5
313/313 [==============================] - 256s - loss: 0.0402 - acc: 0.9845 - val_loss: 0.0605 - val_acc: 0.9792
Epoch 3/5
313/313 [==============================] - 257s - loss: 0.0218 - acc: 0.9921 - val_loss: 0.0763 - val_acc: 0.9770
Epoch 4/5
313/313 [==============================] - 257s - loss: 0.0139 - acc: 0.9960 - val_loss: 0.0547 - val_acc: 0.9808
Epoch 5/5
313/313 [==============================] - 257s - loss: 0.0077 - acc: 0.9977 - val_loss: 0.0766 - val_acc: 0.9820


In [6]:
fine_tuning(ResNet50,(224,224),"w_resnet_00.h5")

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
200/200 [==============================] - 249s - loss: 0.3067 - acc: 0.8696 - val_loss: 0.1165 - val_acc: 0.9646
Epoch 2/5
200/200 [==============================] - 246s - loss: 0.1417 - acc: 0.9466 - val_loss: 0.0882 - val_acc: 0.9688
Epoch 3/5
200/200 [==============================] - 247s - loss: 0.1121 - acc: 0.9573 - val_loss: 0.0770 - val_acc: 0.9740
Epoch 4/5
200/200 [==============================] - 247s - loss: 0.0978 - acc: 0.9633 - val_loss: 0.0702 - val_acc: 0.9746
Epoch 5/5
200/200 [==============================] - 247s - loss: 0.0903 - acc: 0.9657 - val_loss: 0.0744 - val_acc: 0.9734


In [17]:
fine_tuning(ResNet50,(224,224),"w_resnet_11.h5",11)

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
313/313 [==============================] - 265s - loss: 0.0820 - acc: 0.9673 - val_loss: 0.0715 - val_acc: 0.9762
Epoch 2/5
313/313 [==============================] - 261s - loss: 0.0328 - acc: 0.9871 - val_loss: 0.0644 - val_acc: 0.9816
Epoch 3/5
313/313 [==============================] - 262s - loss: 0.0143 - acc: 0.9952 - val_loss: 0.0804 - val_acc: 0.9808
Epoch 4/5
313/313 [==============================] - 262s - loss: 0.0087 - acc: 0.9971 - val_loss: 0.0778 - val_acc: 0.9824
Epoch 5/5
313/313 [==============================] - 262s - loss: 0.0058 - acc: 0.9980 - val_loss: 0.0965 - val_acc: 0.9784


In [18]:
fine_tuning(ResNet50,(224,224),"w_resnet_15.h5",15)

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
1000/1000 [==============================] - 323s - loss: 0.0928 - acc: 0.9644 - val_loss: 0.0627 - val_acc: 0.9790
Epoch 2/5
1000/1000 [==============================] - 323s - loss: 0.0494 - acc: 0.9821 - val_loss: 0.0625 - val_acc: 0.9790
Epoch 3/5
1000/1000 [==============================] - 322s - loss: 0.0336 - acc: 0.9883 - val_loss: 0.0804 - val_acc: 0.9790
Epoch 4/5
1000/1000 [==============================] - 322s - loss: 0.0243 - acc: 0.9925 - val_loss: 0.0797 - val_acc: 0.9802
Epoch 5/5
1000/1000 [==============================] - 328s - loss: 0.0143 - acc: 0.9956 - val_loss: 0.1436 - val_acc: 0.9652


In [14]:
fine_tuning(Xception,(299,299),"w_Xception_0.h5",0)

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
1000/1000 [==============================] - 1754s - loss: 0.0392 - acc: 0.9877 - val_loss: 0.0544 - val_acc: 0.9844
Epoch 2/5
1000/1000 [==============================] - 1749s - loss: 0.0083 - acc: 0.9974 - val_loss: 0.0561 - val_acc: 0.9884
Epoch 3/5
1000/1000 [==============================] - 1751s - loss: 0.0024 - acc: 0.9992 - val_loss: 0.0338 - val_acc: 0.9934
Epoch 4/5
1000/1000 [==============================] - 1752s - loss: 6.8063e-04 - acc: 0.9996 - val_loss: 0.0446 - val_acc: 0.9918
Epoch 5/5
1000/1000 [==============================] - 1750s - loss: 0.0014 - acc: 0.9996 - val_loss: 0.0296 - val_acc: 0.9942


In [26]:
fine_tuning(Xception,(299,299),"w_Xception_00.h5")

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
1000/1000 [==============================] - 638s - loss: 0.1814 - acc: 0.9531 - val_loss: 0.0725 - val_acc: 0.9858
Epoch 2/5
1000/1000 [==============================] - 631s - loss: 0.0868 - acc: 0.9744 - val_loss: 0.0525 - val_acc: 0.9856
Epoch 3/5
1000/1000 [==============================] - 631s - loss: 0.0759 - acc: 0.9748 - val_loss: 0.0453 - val_acc: 0.9866
Epoch 4/5
1000/1000 [==============================] - 632s - loss: 0.0698 - acc: 0.9761 - val_loss: 0.0419 - val_acc: 0.9874
Epoch 5/5
1000/1000 [==============================] - 631s - loss: 0.0608 - acc: 0.9799 - val_loss: 0.0389 - val_acc: 0.9880


In [4]:
fine_tuning(Xception,(299,299),"w_Xception_00_prep.h5",0, xception.preprocess_input)

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
200/200 [==============================] - 740s - loss: 0.2993 - acc: 0.9216 - val_loss: 0.1424 - val_acc: 0.9838
Epoch 2/5
200/200 [==============================] - 629s - loss: 0.1183 - acc: 0.9771 - val_loss: 0.0846 - val_acc: 0.9848
Epoch 3/5
200/200 [==============================] - 629s - loss: 0.0813 - acc: 0.9818 - val_loss: 0.0661 - val_acc: 0.9848
Epoch 4/5
200/200 [==============================] - 638s - loss: 0.0652 - acc: 0.9843 - val_loss: 0.0524 - val_acc: 0.9876
Epoch 5/5
200/200 [==============================] - 638s - loss: 0.0583 - acc: 0.9842 - val_loss: 0.0473 - val_acc: 0.9886


In [8]:
fine_tuning(Xception,(299,299),"w_Xception_10.h5",10)

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
200/200 [==============================] - 654s - loss: 0.0536 - acc: 0.9851 - val_loss: 0.0262 - val_acc: 0.9906
Epoch 2/5
200/200 [==============================] - 651s - loss: 0.0178 - acc: 0.9941 - val_loss: 0.0274 - val_acc: 0.9898
Epoch 3/5
200/200 [==============================] - 651s - loss: 0.0130 - acc: 0.9953 - val_loss: 0.0278 - val_acc: 0.9898
Epoch 4/5
200/200 [==============================] - 651s - loss: 0.0079 - acc: 0.9978 - val_loss: 0.0259 - val_acc: 0.9906
Epoch 5/5
200/200 [==============================] - 651s - loss: 0.0053 - acc: 0.9985 - val_loss: 0.0423 - val_acc: 0.9866


In [20]:
fine_tuning(InceptionV3,(299,299),"w_InceptionV3_0.h5",0)

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
1000/1000 [==============================] - 1115s - loss: 0.0737 - acc: 0.9726 - val_loss: 0.5499 - val_acc: 0.8926
Epoch 2/5
1000/1000 [==============================] - 1102s - loss: 0.0353 - acc: 0.9864 - val_loss: 0.0619 - val_acc: 0.9840
Epoch 3/5
1000/1000 [==============================] - 1101s - loss: 0.0206 - acc: 0.9932 - val_loss: 0.1323 - val_acc: 0.9642
Epoch 4/5
1000/1000 [==============================] - 1101s - loss: 0.0122 - acc: 0.9961 - val_loss: 0.0442 - val_acc: 0.9872
Epoch 5/5
1000/1000 [==============================] - 1102s - loss: 0.0111 - acc: 0.9963 - val_loss: 0.0400 - val_acc: 0.9862


In [4]:
fine_tuning(InceptionV3,(299,299),"w_InceptionV3_00.h5")

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
200/200 [==============================] - 373s - loss: 0.3422 - acc: 0.8769 - val_loss: 0.1631 - val_acc: 0.9688
Epoch 2/5
200/200 [==============================] - 370s - loss: 0.1639 - acc: 0.9526 - val_loss: 0.1099 - val_acc: 0.9734
Epoch 3/5
200/200 [==============================] - 370s - loss: 0.1232 - acc: 0.9619 - val_loss: 0.0885 - val_acc: 0.9780
Epoch 4/5
200/200 [==============================] - 371s - loss: 0.1088 - acc: 0.9640 - val_loss: 0.0821 - val_acc: 0.9768
Epoch 5/5
200/200 [==============================] - 373s - loss: 0.0988 - acc: 0.9666 - val_loss: 0.0739 - val_acc: 0.9782


In [5]:
fine_tuning(InceptionV3,(299,299),"w_InceptionV3_00.h5",0, inception_v3.preprocess_input)

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
200/200 [==============================] - 391s - loss: 0.3740 - acc: 0.8482 - val_loss: 0.1660 - val_acc: 0.9756
Epoch 2/5
200/200 [==============================] - 388s - loss: 0.1681 - acc: 0.9536 - val_loss: 0.1172 - val_acc: 0.9758
Epoch 3/5
200/200 [==============================] - 388s - loss: 0.1262 - acc: 0.9618 - val_loss: 0.0908 - val_acc: 0.9778
Epoch 4/5
200/200 [==============================] - 388s - loss: 0.1072 - acc: 0.9652 - val_loss: 0.0778 - val_acc: 0.9798
Epoch 5/5
200/200 [==============================] - 388s - loss: 0.1009 - acc: 0.9653 - val_loss: 0.0773 - val_acc: 0.9788


In [5]:
fine_tuning(InceptionV3,(299,299),"w_InceptionV3_22.h5",22)

(?, 2048)
(?, 1)
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
200/200 [==============================] - 521s - loss: 0.0665 - acc: 0.9744 - val_loss: 0.0504 - val_acc: 0.9830
Epoch 2/5
200/200 [==============================] - 386s - loss: 0.0345 - acc: 0.9877 - val_loss: 0.0400 - val_acc: 0.9854
Epoch 3/5
200/200 [==============================] - 386s - loss: 0.0269 - acc: 0.9902 - val_loss: 0.0463 - val_acc: 0.9860
Epoch 4/5
200/200 [==============================] - 386s - loss: 0.0219 - acc: 0.9928 - val_loss: 0.0390 - val_acc: 0.9860
Epoch 5/5
200/200 [==============================] - 386s - loss: 0.0170 - acc: 0.9947 - val_loss: 0.0427 - val_acc: 0.9852
